In [ ]:
!rm -r /content/logs/*

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/scalars

In [ ]:
import tensorflow as tf

# download and load dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_validation, y_validation) = mnist.load_data()

# transform images to vectors (flatenning) and change type
x_train = x_train.reshape(60000, 784)
x_validation = x_validation.reshape(10000, 784)
x_train = x_train.astype('float32')
x_validation = x_validation.astype('float32')

# normalize vectors values from [0,255] -> [0,1]
x_train /= 255
x_validation /= 255

print('train samples', x_train.shape)
print('test samples', x_validation.shape)

print('train label samples', y_train.shape)
print('test label samples', y_validation.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

# convert labels from category to one-hot
y_train = to_categorical(y_train, num_classes=10)
y_validation = to_categorical(y_validation, num_classes=10)

In [ ]:
N_EPOCH = 100

# define each model training configuration
experiments = [

    # base
    {'epoch': N_EPOCH, 'optimizer': tf.keras.optimizers.SGD, 'lr': 0.1, 'layers': [128, 64, 32], 'activation': 'relu',
     'batch_size': 16, 'dropout': 0.5, 'BN': True},

    # no dropout
    {'epoch': N_EPOCH, 'optimizer': tf.keras.optimizers.SGD, 'lr': 0.1, 'layers': [128, 64, 32], 'activation': 'relu',
     'batch_size': 16, 'dropout': 0.0, 'BN': True},

    # deeper network
    {'epoch': N_EPOCH, 'optimizer': tf.keras.optimizers.SGD, 'lr': 0.1, 'layers': [128, 64, 32, 16], 'activation': 'relu',
     'batch_size': 16, 'dropout': 0.5, 'BN': True},

    # batch size of 32
    {'epoch': N_EPOCH, 'optimizer': tf.keras.optimizers.SGD, 'lr': 0.1, 'layers': [128, 64, 32], 'activation': 'relu',
     'batch_size': 32, 'dropout': 0.5, 'BN': True},

    # batch size of 64
    {'epoch': N_EPOCH, 'optimizer': tf.keras.optimizers.SGD, 'lr': 0.1, 'layers': [128, 64, 32], 'activation': 'relu',
     'batch_size': 64, 'dropout': 0.5, 'BN': True},

    # activation function sigmoid
    {'epoch': N_EPOCH, 'optimizer': tf.keras.optimizers.SGD, 'lr': 0.1, 'layers': [128, 64, 32], 'activation': 'sigmoid',
     'batch_size': 16, 'dropout': 0.5, 'BN': True},

    # no batch normalization
    {'epoch': N_EPOCH, 'optimizer': tf.keras.optimizers.SGD, 'lr': 0.1, 'layers': [128, 64, 32], 'activation': 'relu',
     'batch_size': 16, 'dropout': 0.5, 'BN': False},

    # smaller learning rate
    {'epoch': N_EPOCH, 'optimizer': tf.keras.optimizers.SGD, 'lr': 0.01, 'layers': [128, 64, 32], 'activation': 'relu',
     'batch_size': 16, 'dropout': 0.5, 'BN': True},

    # optimizer Adam
    {'epoch': N_EPOCH, 'optimizer': tf.keras.optimizers.Adam, 'lr': 0.1, 'layers': [128, 64, 32], 'activation': 'relu',
     'batch_size': 16, 'dropout': 0.5, 'BN': True},
]

In [ ]:
MODELS = {}
for i, config in enumerate(experiments):

    # specify tensorboard directory
    logdir = "logs/scalars/" + str(i)

    # create tensorboard callback
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

    # create saving model callback
    checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='model_{}.ckpt'.format(str(i)), monitor='val_loss', verbose=1, save_best_only=True)

    # define model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(config['layers'][0], activation=config['activation'], input_dim=784))
    for hiddden_dim in config['layers'][1:]:
        if config['BN']:
            model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(config['dropout']))
        model.add(tf.keras.layers.Dense(hiddden_dim, activation=config['activation']))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    # define optimizer
    optim = config['optimizer'](lr=config['lr'])

    # compile model with optimizer, loss and metrics
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

    # train model
    history = model.fit(x_train, y_train, validation_split=0.2, batch_size=config['batch_size'], epochs=config['epoch'], verbose=1, callbacks=[tensorboard_callback, checkpointer])

    # add model to dictionary of models
    MODELS.update({'experiment_{}'.format(str(i)): (history, model)})

In [ ]:
# select model with best validation loss
# the name should be set according to the validation accuracy
history, model = MODELS['experiment_0']   # change name of model based on best validation accuracy

In [ ]:
import numpy as np
import random
from PIL import Image
from IPython.display import display

# display best model prediction on validation examples
for _ in range(10):

    # display example
    idx = random.choice(range(x_validation.shape[0]))
    image = x_validation[idx].reshape(28, 28)
    image = (image * 255).astype(dtype=np.uint8)
    display(Image.fromarray(image).resize((128,128)))

    # display model prediction
    datapoint = x_validation[idx].reshape(1, 784)
    predict_prob = model.predict(datapoint, verbose=0)
    print('i am confident around {:.2f}% that this image corresponds to digit {}'.format(np.amax(predict_prob)*100, np.argmax(predict_prob)))

In [ ]:
x_validation = x_validation.reshape(10000, 784)
loss, acc = model.evaluate(x_validation, y_validation, batch_size=64, verbose=1)
print('test accuracy: ', round(100*acc, 2))